In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tensile/hardness-tensile data.csv
/kaggle/input/test-data-60/final_test_dataset.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from learntools.core import binder
binder.bind(globals())
from learntools import *
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from numpy import absolute
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
from numpy import mean
from sklearn.metrics import mean_squared_error
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import ExtraTreesClassifier
import warnings
import pandas as pd


In [3]:
df = pd.read_csv('/kaggle/input/tensile/hardness-tensile data.csv')
df.head()

,Sr. no,specimen number,Voltage,distance,Gas flow rate,Hardness Test HBR Number,Tensile Test Mpa,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1.0,43 58,25.0,1.0,5.0,75.10,356.667,NaN,NaN,NaN,NaN
1,2.0,56 57,30.0,0.0,5.0,54.95,255.139,NaN,NaN,NaN,NaN
2,3.0,55 54,35.0,0.0,5.0,72.75,250.694,NaN,NaN,NaN,NaN
3,4.0,51 53,40.0,1.0,5.0,69.60,201.528,NaN,NaN,NaN,NaN
4,5.0,45 44,20.0,0.0,5.0,60.95,295.278,NaN,NaN,NaN,NaN


In [4]:
x1=df[['Voltage ', 'distance','Gas flow rate']]
x=x1.dropna()

y1=df[['Hardness Test HBR Number', 'Tensile Test Mpa']]
y=y1.dropna()

In [5]:
n_inputs = x.shape[1]
n_outputs = y.shape[1]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.03, random_state=0)
X_train

,Voltage,distance,Gas flow rate
6,30.0,1.0,10.0
8,40.0,1.0,10.0
9,25.0,2.0,15.0
13,30.0,2.0,20.0
4,20.0,0.0,5.0
2,35.0,0.0,5.0
14,35.0,2.0,20.0
10,30.0,0.0,15.0
7,35.0,0.0,10.0
15,40.0,0.0,20.0


In [7]:
n_estimators = [90,120,140] # number of trees in the random forest
max_features = ['auto', 'sqrt'] # number of features in consideration at every split
max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
min_samples_split = [3, 6, 10] # minimum sample number to split a node
min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
bootstrap = [True, False] # method used to sample data points

random_grid = {'n_estimators': n_estimators,

'max_features': max_features,

'max_depth': max_depth,

'min_samples_split': min_samples_split,

'min_samples_leaf': min_samples_leaf,

'bootstrap': bootstrap}

In [8]:
rf = RandomForestRegressor()

In [9]:
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
               n_iter = 100, cv = 3, verbose=2, random_state=90, n_jobs = -1)

In [10]:
rf_random.fit(x, y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=False, max_depth=20, max_features=auto, min_samples_leaf=3, min_samples_split=10, n_estimators=90; total time=   0.1s
[CV] END bootstrap=False, max_depth=80, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=90; total time=   0.1s
[CV] END bootstrap=True, max_depth=70, max_features=sqrt, min_samples_leaf=4, min_samples_split=6, n_estimators=90; total time=   0.2s
[CV] END bootstrap=True, max_depth=60, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=90; total time=   0.2s
[CV] END bootstrap=False, max_depth=70, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=90; total time=   0.1s
[CV] END bootstrap=True, max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=120; total time=   0.3s
[CV] END bootstrap=True, max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=120; 

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 3, 4],
                                        'min_samples_split': [3, 6, 10],
                                        'n_estimators': [90, 120, 140]},
                   random_state=90, verbose=2)

In [11]:
print ('Random grid: ', random_grid, '\n')
# print the best parameters
print ('Best Parameters: ', rf_random.best_params_, ' \n')

Random grid:  {'n_estimators': [90, 120, 140], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120], 'min_samples_split': [3, 6, 10], 'min_samples_leaf': [1, 3, 4], 'bootstrap': [True, False]} 

Best Parameters:  {'n_estimators': 140, 'min_samples_split': 3, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 20, 'bootstrap': False}  



In [12]:
m1 = RandomForestRegressor(n_estimators= 100, min_samples_split= 2, min_samples_leaf= 1, max_features = 'sqrt', max_depth= 100, bootstrap= False)
# define the direct multioutput wrapper model
m2 = MultiOutputRegressor(m1)

In [13]:
m2.fit(x,y)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=False,
                                                     max_depth=100,
                                                     max_features='sqrt'))

In [14]:
y_pred= m2.predict(x)
y_pred

array([[ 75.1  , 356.667],
       [ 54.95 , 255.139],
       [ 72.75 , 250.694],
       [ 69.6  , 201.528],
       [ 60.95 , 295.278],
       [ 61.   , 226.389],
       [ 61.65 , 292.5  ],
       [ 76.15 , 234.583],
       [ 47.65 , 179.722],
       [ 65.1  , 220.556],
       [ 68.6  , 306.945],
       [ 73.   , 205.139],
       [ 72.2  , 197.639],
       [ 65.65 , 246.945],
       [ 45.8  , 270.556],
       [ 63.05 , 257.361]])

In [15]:
cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=None)
# evaluate the model and collect the scores
cv

RepeatedKFold(n_repeats=10, n_splits=10, random_state=None)

In [16]:
n_scores = cross_val_score(m2, x, y_pred, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 22.512 (10.775)


In [17]:
errors = abs(y_pred - y)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Mean Absolute Error: Hardness Test HBR Number    0.0
Tensile Test Mpa            0.0
dtype: float64
Accuracy: Hardness Test HBR Number    100.0
Tensile Test Mpa            100.0
dtype: float64 %.


In [18]:
test = pd.read_csv('/kaggle/input/test-data-60/final_test_dataset.csv')
test.head()

,Voltage,Distance,Gas Flow Rate
0,15,0,5
1,15,1,5
2,15,2,5
3,15,0,10
4,15,1,10


In [19]:
y_pred1= m2.predict(test)
y_pred1

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Distance
- Gas Flow Rate
Feature names seen at fit time, yet now missing:
- Gas flow rate
- distance

  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Distance
- Gas Flow Rate
Feature names seen at fit time, yet now missing:
- Gas flow rate
- distance

  warnings.warn(message, FutureWarning)


array([[ 60.95   , 295.278  ],
       [ 69.2985 , 343.16142],
       [ 62.955  , 310.51692],
       [ 65.041  , 275.42929],
       [ 62.8185 , 280.73064],
       [ 60.982  , 233.2779 ],
       [ 65.0525 , 292.40047],
       [ 67.0825 , 259.13637],
       [ 63.689  , 228.0282 ],
       [ 64.2995 , 282.16152],
       [ 65.748  , 262.78087],
       [ 64.066  , 242.54215],
       [ 60.95   , 295.278  ],
       [ 69.2985 , 343.16142],
       [ 62.955  , 310.51692],
       [ 65.041  , 275.42929],
       [ 62.8185 , 280.73064],
       [ 60.982  , 233.2779 ],
       [ 65.0525 , 292.40047],
       [ 67.0825 , 259.13637],
       [ 63.689  , 228.0282 ],
       [ 64.2995 , 282.16152],
       [ 65.748  , 262.78087],
       [ 64.066  , 242.54215],
       [ 63.326  , 306.75302],
       [ 75.1    , 356.667  ],
       [ 65.371  , 318.88638],
       [ 67.035  , 274.54873],
       [ 64.4855 , 279.70563],
       [ 61.     , 226.389  ],
       [ 67.94   , 291.71994],
       [ 70.435  , 258.31139],
       [

In [20]:
y_pred1.min()

47.65